In [14]:
import geopandas as gpd
from shapely.geometry import Point

from sedona.register import SedonaRegistrator
from pyspark.sql import SparkSession
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.sql.types import GeometryType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.\
    builder.\
    appName("apache-sedona").\
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.0-incubating,org.postgresql:postgresql:42.2.23,org.datasyslab:geotools-wrapper:geotools-24.0") . \
    master("local[*]").\
    getOrCreate()

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark-3.0.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2645c16a-7612-41ec-8bdd-5397b60cf11c;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.14.3 in central
	found com.fasterxml.jackson.core#jackson-databind;2.10.0 in central
	found com.fasterxml.jackson.core#jackson-core;2.10.0 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.10.0.pr1 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.0.0-i

In [4]:
SedonaRegistrator.registerAll(spark)

True

In [15]:
import geopandas as gpd
from shapely.geometry import Point

from sedona.sql.types import GeometryType
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import IntegerType

point_data = [
    [1, Point(21, 52)],
    [2, Point(21, 52)],
    [3, Point(21, 52)],
    [4, Point(21, 52)],
    [5, Point(21, 52)],
    [6, Point(21, 52)]
]

gdf = gpd.GeoDataFrame(point_data, columns=["id", "geom"])

schema = StructType(
    [
        StructField("id", IntegerType(), False),
        StructField("geom", GeometryType(), False),
        
    ]
)

spark_gdf = spark.createDataFrame(gdf, schema)

spark_gdf.printSchema()

In [19]:
# From spark to geopandas

In [20]:
point_data = [[1, "POINT(21 52)"], [2, "POINT(21 52)"]]

spark_gdf = spark.createDataFrame(point_data).\
    selectExpr("_1 AS id", "ST_GeomFromText(_2) AS geom")

pd_dataframe = spark_gdf.toPandas()

gdf = gpd.GeoDataFrame(pd_dataframe)

In [28]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

In [23]:
spark_gdf.show()

+---+-------------+
| id|         geom|
+---+-------------+
|  1|POINT (21 52)|
|  2|POINT (21 52)|
+---+-------------+

